### NLP Research Internship Assignment Biomedical Text Analysis
*data_extraction_starter.ipynb*

In [4]:
# Import necessary libraries
from Bio import Entrez
import ssl

# Bypass SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
# Function to fetch abstracts from PubMed using MeSH terms
def fetch_abstracts(term, max_results=1000):
    """
    Fetch abstracts from PubMed based on search terms.
    
    Parameters:
    term (str): Search term or MeSH term for querying PubMed.
    max_results (int): Maximum number of results to fetch.
    
    Returns:
    list: A list of abstracts fetched from PubMed.
    """
    
    # Provide contact email for Entrez
    Entrez.email = "info@toxgensolutions.eu"
    
    # Perform the search query using Entrez
    handle = Entrez.esearch(db="pubmed", term=term, retmax=max_results)
    
    # Read search results
    record = Entrez.read(handle)
    handle.close()
    
    # Extract PubMed IDs from the search results
    id_list = record["IdList"]
    
    # Check if search returned results
    if not id_list:
        print("No results found.")
        return []
    
    # Fetch abstracts based on PubMed IDs
    handle = Entrez.efetch(db="pubmed", id=id_list, rettype="abstract", retmode="text")


    
    # Read and split the abstracts
    abstracts = handle.read().split("\n\n")
    handle.close()
    
    return abstracts

In [6]:
# Define the search term, e.g., "Cancer Immunotherapy"
search_term = "Cancer Immunotherapy"

# Fetch abstracts using the search term
abstracts = fetch_abstracts(search_term)

# Display first 5 abstracts for quick inspection (optional)
print("First 5 abstracts:\n")
for i, abstract in enumerate(abstracts[:5]):
    print(f"{i+1}. {abstract}\n")

First 5 abstracts:

1. 1. Biomark Med. 2023 Sep 15. doi: 10.2217/bmm-2023-0202. Online ahead of print.

2. CD39 (ENTPD1) in tumors: a potential therapeutic target and prognostic 
biomarker.

3. Li C(1), Zhang L(1), Jin Q(1), Jiang H(1), Wu C(1).

4. Author information:
(1)Department of Hematology, Lanzhou University Second Hospital, Lanzhou, 
730000, China.

5. As a regulator of the dynamic balance between immune-activated extracellular ATP 
and immunosuppressive adenosine, CD39 ectonucleotidase impairs the ability of 
immune cells to exert anticancer immunity and plays an important role in the 
immune escape of tumor cells within the tumor microenvironment. In addition, 
CD39 has been studied in cancer patients to evaluate the prognosis, the efficacy 
of immunotherapy (e.g., PD-1 blockade) and the prediction of recurrence. This 
article reviews the importance of CD39 in tumor immunology, summarizes the 
preclinical evidence on targeting CD39 to treat tumors and focuses on the 
potenti

### Currently, the code provided outputs the first i parts of an element instead of first i abstracts. Therefore, I will adjust it to output the first i abstracts.

In [7]:
# Function to fetch abstracts from PubMed using MeSH terms
def fetch_abstracts_modified(term, max_results=1000):
    """
    Fetch abstracts from PubMed based on search terms.
    
    Parameters:
    term (str): Search term or MeSH term for querying PubMed.
    max_results (int): Maximum number of results to fetch.
    
    Returns:
    list: A list of abstracts fetched from PubMed.
    """
    
    # Provide contact email for Entrez
    Entrez.email = "info@toxgensolutions.eu"
    
    # Perform the search query using Entrez
    handle = Entrez.esearch(db="pubmed", term=term, retmax=max_results)
    
    # Read search results
    record = Entrez.read(handle)
    handle.close()
    
    # Extract PubMed IDs from the search results
    id_list = record["IdList"]
    
    # Check if search returned results
    if not id_list:
        print("No results found.")
        return []
    
    # Fetch abstracts based on PubMed IDs
    handle = Entrez.efetch(db="pubmed", id=id_list, rettype="abstract", retmode="text")
    

    
    # Read and split the abstracts
    abstracts = handle.read().split("\n\n")



    # Initialize a list to store the abstracts and a counter to keep track of which result
    abstracts_text = []
    counter = 1
    # Iterate through the abstracts to see the beginning of a new abstract 
    for i, abstract in enumerate(abstracts):
        # Check if an abstract begins with i. where i is article number

        if abstract.strip().startswith(str(counter) + '.'):
            
            #increment the counter to keep track of article number
            counter+=1
            # Check if there are 5 paragraphs left in the list
            if i + 5 <= len(abstracts):
                
                # Extract the 5th paragraph after the title as the abstract as results follows this order
                abstract = abstracts[i + 4].strip()
                abstracts_text.append(abstract)

                
    handle.close()
    
    return abstracts_text

In [8]:
search_term = "Cancer Immunotherapy"

# Fetch abstracts using the search term
abstracts = fetch_abstracts_modified(search_term)

# Display first 5 abstracts for quick inspection (optional)
print("First 5 abstracts:\n")
for i, abstract in enumerate(abstracts[:5]):
    print(f"{i+1}. {abstract}\n")

First 5 abstracts:

1. As a regulator of the dynamic balance between immune-activated extracellular ATP 
and immunosuppressive adenosine, CD39 ectonucleotidase impairs the ability of 
immune cells to exert anticancer immunity and plays an important role in the 
immune escape of tumor cells within the tumor microenvironment. In addition, 
CD39 has been studied in cancer patients to evaluate the prognosis, the efficacy 
of immunotherapy (e.g., PD-1 blockade) and the prediction of recurrence. This 
article reviews the importance of CD39 in tumor immunology, summarizes the 
preclinical evidence on targeting CD39 to treat tumors and focuses on the 
potential of CD39 as a biomarker to evaluate the prognosis and the response to 
immune checkpoint inhibitors in tumors.

2. Brain tumors are the most common solid tumor in children and the leading cause 
of cancer-related deaths. Over the last few years, improvements have been made 
in the diagnosis and treatment of children with Central Nervous 

#### Now we are set to go let's use some NLP techniques to make use of the abstracts!

# Entity Recognition

In [4]:
pip install scispacy==0.4.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 44 kB 3.1 MB/s eta 0:00:011
     |████████████████████████████████| 5.9 MB 11.9 MB/s eta 0:00:01
  Using cached conllu-4.5.3-py2.py3-none-any.whl (16 kB)
  Using cached nmslib-2.1.1.tar.gz (188 kB)
     |████████████████████████████████| 126 kB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 586 kB 44.3 MB/s eta 0:00:01
  Using cached catalogue-2.0.9-py3-none-any.whl (17 kB)
  Using cached srsly-2.4.7-cp39-cp39-macosx_11_0_arm64.whl (491 kB)
     |████████████████████████████████| 82 kB 5.8 MB/s  eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Library/Developer/CommandLineTools/usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/2r/vrc_nw5s361_wltp9xrbngch0000gn/T/pip-install-_1hlsxnd/nmslib_fa1ec9e1d20f4c248da8c6d4cc360e27/setup.py'"'"'; __file__='"'"'/private/var/folders/2r/vr

In [2]:
import scispacy

ModuleNotFoundError: No module named 'scispacy'

In [2]:
import spacy
from spacy import displacy
import scispacy

NER = spacy.load("en_core_sci_scibert")

/Users/alialsaeedi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'scispacy'

In [ ]:
test = "Hello my name is Ali"
entities = NER(abstracts[1])

for word in entities.ents:
    print(word.text)
    print(word.label_)
    print(" ")

the last few years
DATE
 
Central Nervous System
LOC
 
